In [2]:
pwd

'/home/aditya/LP IV/A5'

In [30]:
data = open(r"/home/aditya/LP IV/untitled.txt")
data

<_io.TextIOWrapper name='/home/aditya/LP IV/untitled.txt' mode='r' encoding='UTF-8'>

In [31]:
corona_data = [text for text in data if text.count(' ') >= 2]
corona_data

["The speed of transmission is an important point of difference between the two viruses. Influenza has a shorter median incubation period (the time from infection to appearance of symptoms) and a shorter serial interval (the time between successive cases) than COVID-19 virus. The serial interval for COVID-19 virus is estimated to be 5-6 days, while for influenza virus, the serial interval is 3 days. This means that influenza can spread faster than COVID-19. \\n',\n",
 " 'Further, transmission in the first 3-5 days of illness, or potentially pre-symptomatic transmission â€“transmission of the virus before the appearance of symptoms â€“ is a major driver of transmission for influenza. In contrast, while we are learning that there are people who can shed COVID-19 virus 24-48 hours prior to symptom onset, at present, this does not appear to be a major driver of transmission. \\n',\n",
 " 'The reproductive number â€“ the number of secondary infections generated from one infected individual 

In [32]:
import numpy as np
import keras.backend as K #imp
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
from keras.utils import np_utils
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
import gensim

In [33]:
vectorize = Tokenizer()
vectorize.fit_on_texts(corona_data)
corona_data = vectorize.texts_to_sequences(corona_data)

total_vocab = sum(len(s) for s in corona_data)
word_count = len(vectorize.index_word)+1
corona_data

[[1,
  39,
  2,
  8,
  9,
  40,
  41,
  42,
  2,
  43,
  13,
  1,
  44,
  23,
  3,
  45,
  11,
  24,
  46,
  47,
  48,
  1,
  14,
  25,
  49,
  10,
  26,
  2,
  27,
  12,
  11,
  24,
  15,
  16,
  1,
  14,
  13,
  50,
  51,
  17,
  4,
  5,
  6,
  1,
  15,
  16,
  7,
  4,
  5,
  6,
  9,
  52,
  10,
  18,
  19,
  53,
  20,
  28,
  7,
  3,
  6,
  1,
  15,
  16,
  9,
  29,
  20,
  30,
  54,
  31,
  3,
  32,
  55,
  56,
  17,
  4,
  5,
  33],
 [57,
  8,
  34,
  1,
  58,
  29,
  19,
  20,
  2,
  59,
  60,
  61,
  62,
  63,
  8,
  64,
  2,
  1,
  6,
  65,
  1,
  26,
  2,
  27,
  21,
  9,
  11,
  35,
  36,
  2,
  8,
  7,
  3,
  34,
  66,
  28,
  67,
  22,
  68,
  31,
  69,
  22,
  70,
  71,
  32,
  72,
  4,
  5,
  6,
  73,
  74,
  75,
  76,
  10,
  77,
  78,
  79,
  80,
  30,
  81,
  82,
  83,
  10,
  18,
  11,
  35,
  36,
  2,
  8,
  33],
 [84,
  85,
  37,
  21,
  1,
  37,
  2,
  86,
  87,
  88,
  25,
  89,
  90,
  91,
  21,
  9,
  92,
  10,
  18,
  13,
  38,
  12,
  38,
  19,
  7,
  4,
  5,


In [34]:
print(total_vocab)
print(word_count)

200
105


In [35]:
window_size = 2

In [39]:
# Defining utility to generate context word pairs
def cbow_model(data, window_size, total_vocab):
    total_length = window_size*2
    for text in data:
        text_len = len(text)
#         print("zero",text)
        for idx, word in enumerate(text):
#             print("first",idx,word)
            context_word = []
            target   = []            
            begin = idx - window_size
            end = idx + window_size + 1
            context_word.append([
                text[i] 
                for i in range(begin, end) 
                if 0 <= i < text_len 
                and i != idx
            ])
            target.append(word)
#             print("second",context_word,target)
            contextual = pad_sequences(
                context_word, 
                maxlen=total_length
            )
            final_target = np_utils.to_categorical(
                target, 
                total_vocab
            )
#             print("third",contextual,final_target)
            yield(contextual, final_target)            

In [40]:
# Defining the model architecture
model = Sequential()
model.add(
    Embedding(
        input_dim=total_vocab, 
        output_dim=100, 
        input_length=window_size*2
    )
)
model.add(
    Lambda(
        lambda x: K.mean(x, axis=1), 
        output_shape=(100,)
    )
)
model.add(
    Dense(
        total_vocab, 
        activation='softmax'
    )
)

In [41]:
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 4, 100)            20000     
                                                                 
 lambda_3 (Lambda)           (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 200)               20200     
                                                                 
Total params: 40,200
Trainable params: 40,200
Non-trainable params: 0
_________________________________________________________________


In [42]:
model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam'
)

In [43]:
for i in range(10):
    cost = 0
    for x, y in cbow_model(corona_data, window_size, total_vocab):
#         print(x,y)
        cost += model.train_on_batch(x, y)
    print("Epoch ", i,"\t: ", cost)

Epoch  0 	:  1054.0115232467651
Epoch  1 	:  1002.6175694465637
Epoch  2 	:  912.5089628696442
Epoch  3 	:  836.2285907268524
Epoch  4 	:  782.3472242355347
Epoch  5 	:  730.7972129583359
Epoch  6 	:  676.9568408727646
Epoch  7 	:  621.16617000103
Epoch  8 	:  564.8284534215927
Epoch  9 	:  509.7091538310051


In [46]:
dimensions = 100
vect_file = open('./vectors.txt','w')
vect_file.write('{} {}\n'.format(102, dimensions))


8

In [47]:
weights = model.get_weights()[0]
for text, i in vectorize.word_index.items():
    final_vec = ' '.join(map(str, list(weights[i, :])))
    vect_file.write('{} {}\n'.format(text, final_vec))
vect_file.close()

In [48]:
cbow_output = gensim.models.KeyedVectors.load_word2vec_format(
    'vectors.txt', 
    binary=False
)

In [49]:
cbow_output.most_similar(positive=['speed'])


[('â€“transmission', 0.8413423299789429),
 ('difference', 0.8297915458679199),
 ('two', 0.7281671166419983),
 ("n'", 0.7226202487945557),
 ('driver', 0.7114925384521484),
 ('of', 0.6631213426589966),
 ('serial', 0.6516485810279846),
 ('before', 0.6498878598213196),
 ('number', 0.6427524089813232),
 ('appearance', 0.6317931413650513)]